***
# Homework 2

**Course:** MATH 616 - Data-Driven Dynamical Systems, Stochastic Modeling and Prediction

**Name:** Shrivats Sudhir

**NetID:** ssudhir2

**Email:** ssudhir2@wisc.edu

***

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm

# Question 1.

**Assume $x_t$ satisfies the SDE:**
$$d x_t = a_t\;dt + b_t\;dW_t,$$

## (a.)

**Derive the SDE of $f(x_t)$ when $f(x_t)=\frac{1}{2}x^2_t$**

## (b.)

**Derive the SDE of $f(W_t)$ when $f(W_t)=W^4_t$**

# Question 2.

**Consider a two-state Markov chain with state space $S=\{1,2\}$ and transition probabilities $p=0.125$ and $q=0.25$. Numerically generate a long time series, using which to compute the expectation, the probably at each state, and the $n$-step transition probability matrix for $n=10$ and $n=100$. Compare your numerical results with the theoretical solutions.**

# Question 3.

**Use Monte Carlo simulations to compute the mean of a one-dimensional standard Gaussian distribution (with zero mean and variance being $1$). How many Monte Carlo sample points is needed if the standard error is less than $0.05$?**

# Question 4.

**Please use Monte Carlo simulation to compute $\sqrt{2}$. Show the standard error as a function of number of samples.**